In [1]:
import os
import tensorflow as tf

from tensorflow.python.framework import ops

In [2]:
meta_path = '/Users/victor/Project/research/facerecognition/bin/20170512-110547/model-20170512-110547.meta'
ckpt_path = '/Users/victor/Project/research/facerecognition/bin/20170512-110547/model-20170512-110547.ckpt-250000'

with tf.Graph().as_default(): 
    with tf.Session().as_default() as sess:
        saver = tf.train.import_meta_graph(meta_path)
        saver.restore(sess, ckpt_path)
        print('model loaded')

In [17]:
op = sess.graph.get_operations()
[m.values() for m in op if 'InceptionResnetV1' in m.name and 'weights' in m.name]

[(<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal/shape:0' shape=(4,) dtype=int32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal/mean:0' shape=() dtype=float32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal/stddev:0' shape=() dtype=float32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal/TruncatedNormal:0' shape=(3, 3, 3, 32) dtype=float32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal/mul:0' shape=(3, 3, 3, 32) dtype=float32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Initializer/truncated_normal:0' shape=(3, 3, 3, 32) dtype=float32>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/weights/Assign:0' shape=(3, 3, 3, 32) dtype=float32_ref>,),
 (<tf.Tensor 'InceptionResnetV1/Conv2d_1a_3x3/

In [1]:
from image_util import *
import tensorflow as tf
from tensorflow.python.ops import data_flow_ops
import numpy as np

data_dir = '/Users/victor/Downloads/flower_photos'

dataset = get_data(data_dir)
image_list, label_list = get_image_list_with_labels(dataset)
print('elements:', len(image_list))

images = np.expand_dims(np.array(image_list), 1)
labels = np.expand_dims(np.array(label_list), 1)

image_paths_placeholder = tf.placeholder(tf.string, shape=(None,1), name='image_paths')
labels_placeholder = tf.placeholder(tf.int64, shape=(None,1), name='labels')
        
input_queue = data_flow_ops.FIFOQueue(capacity=100000,
                           dtypes=[tf.string, tf.int64],
                           shapes=[(1,), (1,)],
                           shared_name=None, name=None)
enqueue_op = input_queue.enqueue_many([image_paths_placeholder, labels_placeholder], name='enqueue_op')

image_batch, label_batch = read_images_batch(input_queue, 32, 160)
image_batch = tf.Print(image_batch, data=[input_queue.size()], message='Nb elements left:')

with tf.Session() as sess: 
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    sess.run(enqueue_op, feed_dict={image_paths_placeholder: images, labels_placeholder: labels})
    
    for i in range(114):
        print(i, sess.run(image_batch).shape)
    print('done')
        
    coord.request_stop()
    coord.join(threads)

elements: 3670


TypeError: shuffle_batch_join() missing 1 required positional argument: 'min_after_dequeue'